In [2]:
import pandas as pd

# Read the CSV file
file_path = '../preped.csv'
df = pd.read_csv(file_path)

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


# Select features and target
features = ['Hidden Gem Score', 'Runtime', 'Awards Received', 'Awards Nominated For',
           'Boxoffice', 'IMDb Votes', 'Minimum Age'] + \
           [col for col in df.columns if col in ['Action', 'Adventure', 'Animation', 
           'Biography', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Family', 
           'Fantasy', 'History', 'Horror', 'Music', 'Musical', 'Mystery', 
           'News', 'Romance', 'Sci-Fi', 'Sport', 'Thriller', 'War', 'Western']]

target = 'IMDb Score'

# Preprocessing
X = df[features]
y = df[target]

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Create and train SVM model with RBF kernel
model = SVR(kernel='rbf', C=1.0, epsilon=0.1)
model.fit(X_train, y_train)

# Evaluate model
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f'Mean Absolute Error: {mae:.2f}')
print(f'Mean Squared Error: {mse:.2f}')
print(f'R^2 Score: {r2:.3f}')

# Example prediction
example = X_test[0].reshape(1, -1)
prediction = model.predict(example)
print(f'Predicted IMDb Score: {prediction[0]:.2f}')

Mean Absolute Error: 0.37
Mean Squared Error: 0.28
R^2 Score: 0.671
Predicted IMDb Score: 8.31


In [4]:
from sklearn.model_selection import GridSearchCV

# Define parameter grid for SVR
param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': ['scale', 'auto', 0.1, 0.01],
    'epsilon': [0.1, 0.2, 0.5],
    'kernel': ['rbf', 'linear']
}

# Create GridSearchCV object
grid_search = GridSearchCV(
    SVR(),
    param_grid,
    cv=5,
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    verbose=1
)

# Fit grid search
grid_search.fit(X_train, y_train)

# Print best parameters and score
print("Best parameters:", grid_search.best_params_)
print("Best MSE score:", -grid_search.best_score_)

# Use best model to make predictions
best_model = grid_search.best_estimator_
y_pred_best = best_model.predict(X_test)

# Calculate metrics for best model
mae_best = mean_absolute_error(y_test, y_pred_best)
mse_best = mean_squared_error(y_test, y_pred_best)
r2_best = r2_score(y_test, y_pred_best)

print("\nBest Model Metrics:")
print(f'Mean Absolute Error: {mae_best:.2f}')
print(f'Mean Squared Error: {mse_best:.2f}')
print(f'R^2 Score: {r2_best:.2f}')

Fitting 5 folds for each of 96 candidates, totalling 480 fits


KeyboardInterrupt: 

In [ ]:
# Define target columns
target_rt = 'Rotten Tomatoes Score'
target_mc = 'Metacritic Score'

# Prepare data for Rotten Tomatoes Score
y_rt = df[target_rt]
X_train_rt, X_test_rt, y_train_rt, y_test_rt = train_test_split(X, y_rt, test_size=0.2, random_state=42)

# Prepare data for Metacritic Score
y_mc = df[target_mc]
X_train_mc, X_test_mc, y_train_mc, y_test_mc = train_test_split(X, y_mc, test_size=0.2, random_state=42)

# Scale features (reusing existing scaler)
X_train_rt_scaled = scaler.fit_transform(X_train_rt)
X_test_rt_scaled = scaler.transform(X_test_rt)
X_train_mc_scaled = scaler.fit_transform(X_train_mc)
X_test_mc_scaled = scaler.transform(X_test_mc)

# Train models
model_rt = SVR(kernel='rbf', C=1.0, epsilon=0.1)
model_mc = SVR(kernel='rbf', C=1.0, epsilon=0.1)

model_rt.fit(X_train_rt_scaled, y_train_rt)
model_mc.fit(X_train_mc_scaled, y_train_mc)

# Make predictions
y_pred_rt = model_rt.predict(X_test_rt_scaled)
y_pred_mc = model_mc.predict(X_test_mc_scaled)

# Evaluate models
mae_rt = mean_absolute_error(y_test_rt, y_pred_rt)
mse_rt = mean_squared_error(y_test_rt, y_pred_rt)
r2_rt = r2_score(y_test_rt, y_pred_rt)

mae_mc = mean_absolute_error(y_test_mc, y_pred_mc)
mse_mc = mean_squared_error(y_test_mc, y_pred_mc)
r2_mc = r2_score(y_test_mc, y_pred_mc)

# Print results
print("Rotten Tomatoes Score Metrics:")
print(f"Mean Absolute Error: {mae_rt:.2f}")
print(f"Mean Squared Error: {mse_rt:.2f}")
print(f"R^2 Score: {r2_rt:.2f}")

print("\nMetacritic Score Metrics:")
print(f"Mean Absolute Error: {mae_mc:.2f}")
print(f"Mean Squared Error: {mse_mc:.2f}")
print(f"R^2 Score: {r2_mc:.2f}")

Rotten Tomatoes Score Metrics:
Mean Absolute Error: 14.11
Mean Squared Error: 341.64
R^2 Score: 0.51

Metacritic Score Metrics:
Mean Absolute Error: 8.57
Mean Squared Error: 124.45
R^2 Score: 0.59


In [ ]:
import matplotlib.pyplot as plt

# Create a figure with 3 subplots in a vertical arrangement
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5))

# Plot IMDb Score comparison
ax1.scatter(y_test, y_pred, alpha=0.6)
ax1.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
ax1.set_xlabel('Actual IMDb Score')
ax1.set_ylabel('Predicted IMDb Score')
ax1.set_title('Actual vs. Predicted IMDb Scores')

# Plot Rotten Tomatoes Score comparison
ax2.scatter(y_test_rt, y_pred_rt, alpha=0.6)
ax2.plot([y_test_rt.min(), y_test_rt.max()], [y_test_rt.min(), y_test_rt.max()], 'r--', lw=2)
ax2.set_xlabel('Actual Rotten Tomatoes Score')
ax2.set_ylabel('Predicted Rotten Tomatoes Score')
ax2.set_title('Actual vs. Predicted Rotten Tomatoes Scores')

# Plot Metacritic Score comparison
ax3.scatter(y_test_mc, y_pred_mc, alpha=0.6)
ax3.plot([y_test_mc.min(), y_test_mc.max()], [y_test_mc.min(), y_test_mc.max()], 'r--', lw=2)
ax3.set_xlabel('Actual Metacritic Score')
ax3.set_ylabel('Predicted Metacritic Score')
ax3.set_title('Actual vs. Predicted Metacritic Scores')

# Adjust layout to prevent overlap
plt.tight_layout()
plt.show()